In [1]:
%pip install google-genai
%pip install python-dotenv
%pip install beautifulsoup4
%pip install googlesearch-python


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google import genai
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
cx_id = os.getenv("GOOGLE_CSE_ID")
client = genai.Client(api_key=api_key)

Take the input from file 'data/generated_recipes.json' and check each recipe for correctness.

The moderaror will take the question and recipe to prompt the LLM model to check the recipe for correctness.

Now use the medical docs to check each recipe for correctness.


In [3]:
def check_recipe_moderator(recipe):
    """Check the recipe using the RAG model."""
    medical_question = recipe.get('medische_vraag')
    recept = recipe.get('recept')
    patient_info = recipe.get('patientinformatie')

    prompt = f"""
    Controleer het volgende recept op juistheid:

    Vraag: {medical_question}

    Patient informatie: {patient_info}

    Recept: {recept}
    """

    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": {
                "type": "object",
                "properties": {
                    "correct": {"type": "boolean"},
                    "explanation": {"type": "string"}
                },
                "required": ["correct", "explanation"]
            },
            "system_instruction": ("Beoordeel het recept op basis van de volgende criteria: "
                                  "1. Bevat het medicijn een geldige naam, sterkte, vorm, aantal doses en instructies? "
                                  "2. Is de indicatie relevant voor de medische vraag en gegeven patient informatie? "
                                  "3. Geef een beoordeling van het recept als correct = True|False. ")
        }
    )
    return json.loads(response.text)



In [4]:
import json

# Read the JSON file
with open('../data/generated_recipes.json', 'r', encoding='utf-8') as file:
    recipes = json.load(file)

# Loop through each recipe and access the 'medicijn' field
for recipe in recipes:
    medicijn = recipe.get('medicijn')
    # prompt the AI model to check the recipe
    # add the documentation to the prompt
    result = check_recipe_moderator(recipe)
    recipe['correct'] = result['correct']
    recipe['explanation'] = result['explanation']

# Write the recipes to a JSON file
output_file = "../data/generated_recipes_moderator_review.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(recipes, f, ensure_ascii=False, indent=4)
